## 打印可视化界面

In [ ]:
from onekey_algo.datasets.image_loader import default_loader
from onekey_algo.custom.components.comp2 import show_cam_on_image
import torch
import os
import random
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey
from onekey_algo.utils.MultiProcess import MultiProcess
import numpy as np
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.use('Agg')
epoch_mapping = {'resnet101': {'MR-CE_anno': 37, 'MR-CE': 32, 'MR-T2_anno': 48, 'MR-T2': 42, 'endoscope': 25,  'endoscope_anno': 38, },
                 'densenet121': {'MR-CE_anno': 45, 'MR-CE': 23, 'MR-T2_anno': 40, 'MR-T2': 44, 'endoscope': 28, 'endoscope_anno': 38, },
                 'CrossFormer': {'MR-CE_anno': 253, 'MR-CE': 244, 'MR-T2_anno': 285, 'MR-T2': 206,
                                 'endoscope': 141, 'endoscope_anno': 110, },}


def viz_sample(samples, thread_id, model_root, force_epoch):
    model, transformer, device = init_from_onekey(os.path.join(model_root, 'viz'), 
                                                  force_pth=os.path.join(model_root, f'train/training-params-{8*force_epoch}.pth'))
    for n, m in model.named_modules():
        print('Feature name:', n, "|| Module:", m)
    target_layer = "layer4.1.conv2"
    target_layer = 'features.denseblock4.denselayer16.conv2' if 'densenet' in model_root else 'layers.3.1.layers.1.3.4'
#     return 
    gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)
#     return
    viz_dir = os.path.join(model_root, 'Grad-CAM')
    os.makedirs(viz_dir, exist_ok=True)
    for sample in samples:
        if os.path.exists(os.path.join(viz_dir, os.path.basename(sample))):
            continue
        img = default_loader(sample)
        sample_ = transformer(img)
        sample_  = sample_.view(1, *sample_.size()).to(device)
        res_cam = gradcam(x=sample_, class_idx=None)
        fig, axes = plt.subplots(1, 3, figsize=(12, 4), facecolor='white')
    #     axes[0].imshow(-res_cam[0][0].cpu(), cmap='jet')
        axes[0].imshow(img.resize(sample_.size()[2:]))
        axes[0].axis('off')
    #     plt.savefig(f"viz/{os.path.basename(sample).replace('.png', '_se.png')}", bbox_inches = 'tight')
    #     plt.show()
    #     plt.figure(figsize=(10, 10))
    #     plt.axis('off')
        imshow = axes[1].imshow(-res_cam[0][0].cpu(),cmap='jet')
        axes[1].axis('off')
        imshow = axes[2].imshow(show_cam_on_image(img.resize(sample_.size()[2:]), -res_cam[0][0].cpu(), use_rgb=True, reverse=False), 
                                cmap='jet')
        axes[2].axis('off')
        cax = fig.add_axes([0.92, 0.17, 0.02, axes[2].get_position().height]) 
        plt.colorbar(imshow, cax=cax)
        plt.savefig(f'{viz_dir}/{os.path.basename(sample).replace(".npy", ".png")}', bbox_inches = 'tight')
        plt.show()
        plt.close(fig)
for model in ['densenet121',]:
    for modal in epoch_mapping[model].keys():
        if 'endoscope' in modal:
            prefix = 'fcn_crop' if 'anno' in modal else 'crop'
            m = modal.split('_')[0]
            samples = glob(os.path.join(rf'Y:\20240320-RenJiLiang\Segmentation\endoscope_coco\{prefix}', '*'))
        else:
            continue
            prefix = 'vnet_crop' if 'anno' in modal else 'crop'
            samples = glob(os.path.join(rf'Y:\20240320-RenJiLiang\{modal.split("_")[0]}\{prefix}', '*'))
        model_root = rf'Y:\20240320-RenJiLiang\Classification\{modal}\CV-6\{model}'
        force_epoch = epoch_mapping[model][modal]
        random.shuffle(samples)
        viz_sample(samples, thread_id=1, model_root=model_root, force_epoch=force_epoch+1)
# MultiProcess(func=viz_sample, samples=samples, num_process=1).run()